In [33]:
import math
from os import path, walk
from json import dumps, loads
from pyvis.network import Network
from dataclasses import dataclass, field

from attack import Attack
from defend import DTechnique, DSystem, Defend
from types import SimpleNamespace

In [228]:
d = Defend(Attack(local=True))

graph_config = {
    'tactic': {
        'mass': 100,
        'size': 100,
        'color': '#C00'
    },
    'technique': {
        'mass': 50,
        'size': 50,
        'color': '#C77'
    },
    'subtechnique': {
        'mass': 5,
        'size': 20,
        'color': '#A66'
    },
}
gconf = SimpleNamespace(**graph_config)

In [35]:
def add_node(net, name, config):
    net.add_node(name, **config)

In [105]:
Out = './Poligon'

net = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')

In [106]:
# Tactics
prev = None
for tac in d.attack.cfg.tactics_order:
    net.add_node(tac, phyisics=False,label=tac.replace('-', ' ').title(), **gconf.tactic)
    if prev:
        net.add_edge(prev, tac)
        pass
    prev = tac

In [107]:
for tech in d.attack.techniques:
    tac = d.attack.techniques[tech].tactics[0]
    
    parent = d.attack.techniques[tech].get_parent()
    
    if parent and parent not in net.node_ids:
        parent_tac = d.attack.techniques[parent].tactics[0]
        net.add_node(parent, label= d.attack.get_name_by_id(parent), **gconf.technique)
        #net.add_node(parent, color=gconf.technique['color'], mass=gconf.technique['mass'])
        net.add_edge(parent, parent_tac)
    
    if parent:
        net.add_node(tech, label = d.attack.get_name_by_id(tech),  **gconf.subtechnique)
        #net.add_node(tech, color=gconf.subtechnique['color'], mass=gconf.subtechnique['mass'])
        net.add_edge(tech, parent)
    else:
        net.add_node(tech, label = d.attack.get_name_by_id(tech), **gconf.technique)
        #net.add_node(tech, color=gconf.technique['color'], mass=gconf.technique['mass'])
        net.add_edge(tech, tac)

In [49]:
d.attack.techniques['T1542'].tactics

['defense-evasion', 'persistence']

In [109]:
#net.show_buttons()

netcfg = """{
  "edges": {
    "color": {
      "inherit": true
    },
    "selfReference": {
      "angle": 0.7853981633974483
    },
    "smooth": {
      "forceDirection": "none"
    }
  },
  "physics": {
    "barnesHut": {
      "centralGravity": 0.8,
      "springLength": 30,
      "damping": 0.21,
      "avoidOverlap": 0.13
    },
    "minVelocity": 0.75,
    "timestep": 1
  }
}"""
net.set_options(netcfg)
net.show('sigma_multinode_3_sev_Crit.html')

sigma_multinode_3_sev_Crit.html


In [11]:
import yaml

In [43]:
d.attack.techniques['T1055.011'].__dict__.keys()

dict_keys(['name', 'description', 'detection', 'data_sources', 'platforms', 'permissions_required', 'defenses_bypassed', 'id', 'tactics', 'references', 'ref'])

In [ ]:
with open("example.yaml", 'r') as yaml_in, open("example.json", "w") as json_out:
    yaml_object = yaml.safe_load(yaml_in) # yaml_object will be a list or a dict
    json.dump(yaml_object, json_out)

In [29]:
with open("sigma/rules/windows/driver_load/driver_load_win_mal_creddumper.yml", 'r') as f:
    y = yaml.safe_load(f)

In [14]:
def load_sigma_windows():
    ydata = []
    systemspath = "sigma/rules/windows/"
    for root, subdirectories, files in walk(systemspath):
        for file in files:
            if file[-4:] == ".yml":
                filename = path.join(root, file)
                with open(filename, 'r', encoding='utf-8') as fobj:
                    data = yaml.safe_load(fobj)
                    fobj.close()
                    ydata.append(data)
    return ydata

In [15]:
sigma_windows = load_sigma_windows()

In [16]:
len(sigma_windows)

2173

In [21]:
techniques = []
for ele in sigma_windows[0]['tags']:
    if 't1' in ele:
        techniques.append(ele.replace('attack.', '').title())
techniques

['T1003.002', 'T1003.004', 'T1003.001', 'T1003.006']

In [23]:
sigma = []
for sig in sigma_windows:
    techniques = []
    for ele in sig.get('tags', []):
        if 't1' in ele:
            techniques.append(ele.replace('attack.', '').title())
    simple_sigma = {
        'techniques': techniques,
        'id': sig['id'],
        'severity': sig['level'],
        'title': sig['title']
    }
    sigma.append(simple_sigma)

In [30]:
sigma

[{'techniques': ['T1003.002', 'T1003.004', 'T1003.001', 'T1003.006'],
  'id': '06d71506-7beb-4f22-8888-e2e5e2ca7fd8',
  'severity': 'high',
  'title': 'Mimikatz Use'},
 {'techniques': ['T1211', 'T1562.001'],
  'id': '545a5da6-f103-4919-a519-e9aec1026ee4',
  'severity': 'high',
  'title': 'Microsoft Malware Protection Engine Crash'},
 {'techniques': ['T1003.001'],
  'id': 'a18e0862-127b-43ca-be12-1a542c75c7c5',
  'severity': 'high',
  'title': 'Potential Credential Dumping Via WER - Application'},
 {'techniques': [],
  'id': 'e6e88853-5f20-4c4a-8d26-cd469fd8d31f',
  'severity': 'medium',
  'title': 'Ntdsutil Abuse'},
 {'techniques': [],
  'id': '94dc4390-6b7c-4784-8ffc-335334404650',
  'severity': 'medium',
  'title': 'Dump Ntds.dit To Suspicious Location'},
 {'techniques': ['T1203', 'T1068', 'T1211', 'T1212', 'T1210', 'T1499.004'],
  'id': '48d91a3a-2363-43ba-a456-ca71ac3da5c2',
  'severity': 'critical',
  'title': 'Audit CVE Event'},
 {'techniques': ['T1070.004'],
  'id': '9703792d-fd

In [31]:
sigma_wtech = []
for s in sigma:
    if s['techniques']:
        sigma_wtech.append(s)

In [108]:
for s in sigma_wtech:
    #net.add_node(s['id'], label=s['title'], color='#AAF')
    for t in s['techniques']:
        if t in net.node_ids:
            if s['severity'] == 'critical':
                color = sev_colors[s['severity']]
                net.add_node(s['id']+t, label=s['title'], color=color)
                net.add_edge(s['id']+t, t)

In [89]:
sev = set()
for s in sigma_wtech:
    sev.add(s['severity'])
print(sev)


{'low', 'medium', 'high', 'informational', 'critical'}


In [90]:
sev_colors = {
    "critical": "#E76EFA",
    "high": "#AB62DE",
    "medium": "#9E77F5",
    "low": "#6762DE",
    "informational": "#FFF",
}

In [189]:

net2 = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')
net2.add_node('A', physics=False)

net2.add_node('B', physics=False)
net2.add_node('B1', physics=True)
net2.add_node('B2', physics=True)
net2.add_node('C', physics=False)
net2.add_node('C1', physics=True)
net2.add_node('C2', physics=True)
net2.add_node('D', physics=False)
net2.add_edge('A', 'B', physics=False)
net2.add_edge('C1', 'A', physics=True)
net2.add_edge('C1', 'B', physics=True)
net2.add_edge('C1', 'D', physics=True)
net2.add_edge('D', 'C2', physics=True)

index = 0
result =  get_coords(radius=50000, x=0, y=0, minresults=15)
for pos in result:
    net2.add_node(index, x=pos[0], y=pos[1], physics=False)
    index +=1
    
    level2_result = get_coords(radius=3000, x = pos[0], y=pos[1], minresults=12)
    #print(level2_result)
    index2 = 0
    for pos2 in level2_result:
        net2.add_node(str(index)+"-"+str(index2), x=pos2[0], y=pos2[1], physics=False)
        index2 += 1
    
    
net2.show('test.html')

test.html


In [160]:
import math

a = 0
b = 0
r = 5000

#The lower this value the higher quality the circle is with more points generated
stepSize = 0.3

#Generated vertices
positions = []

t = 0
while t < 2 * math.pi:
    positions.append((r * math.cos(t) + a, r * math.sin(t) + b))
    t += stepSize

print(positions)

[(5000.0, 0.0), (4776.68244562803, 1477.6010333066977), (4126.678074548392, 2823.212366975177), (3108.0498413533223, 3916.6345481374165), (1811.788772383368, 4660.195429836132), (353.6860083385145, 4987.474933020272), (-1136.0104734654356, 4869.238154390976), (-2524.230522999288, 4316.046833244369), (-3686.9685777062273, 3377.315902755755), (-4520.360710085305, 2136.8994011691507), (-4949.962483002227, 705.6000402993383), (-4937.398849544325, -788.7284707162388), (-4483.7920816707365, -2212.6022164742585), (-3629.661521000704, -3438.8307959198655), (-2451.304106703501, -4357.878862067939), (-1053.9789971539028, -4887.650588325485), (437.49491719722755, -4980.823044179204), (1889.888713564897, -4629.073411638664), (3173.464379713166, -3863.822437779941), (4173.563924195794, -2753.4277129881957), (4800.851433251828, -1397.0774909946379)]


In [197]:
def get_coords(radius, x, y, minresults):
    coords = []
    stepSize = 6.0
    while len(coords) < minresults:
        coords = get_circle(radius, x, y, stepSize)
        stepSize -= 0.01
    #print(stepSize)
    return coords

def get_circle(radius, x, y, stepSize=6.0):
    coords = []
    t = 0
    while t < 2 * math.pi:
        coords.append((radius * math.cos(t) + x, radius * math.sin(t) + y))
        t += stepSize
    return coords

In [174]:
result = (get_coords(radius=5000, x=0, y=0, minresults=15))

0.3000000000000046


In [175]:
len(result)

16

In [176]:
result

[(5000.0, 0.0),
 (4605.3049700144165, 1947.0917115432735),
 (3483.533546735794, 3586.7804544976457),
 (1811.788772383304, 4660.195429836156),
 (-145.9976115065351, 4997.868015207523),
 (-2080.734182735817, 4546.487134128361),
 (-3686.9685777063214, 3377.3159027556517),
 (-4711.111703343346, 1674.9407507793705),
 (-4991.473878973755, -291.87071713808666),
 (-4483.792081670642, -2212.602216474452),
 (-3268.2181043178816, -3784.0124765397954),
 (-1536.6643498918518, -4758.010369447659),
 (437.4949171975151, -4980.823044179178),
 (2342.583356502156, -4417.273278600623),
 (3877.8293925514577, -3156.3331893613495),
 (4800.851433251929, -1397.0774909942882)]

In [204]:
net3 = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')

# Tactics
prev = None

tactic_coords = get_coords(20000, x=0, y=0, minresults=len(d.attack.cfg.tactics_order)+1)
print(len(tactic_coords))
print(len(d.attack.cfg.tactics_order))
tac_index = 0

for tac in d.attack.cfg.tactics_order:
    net3.add_node(tac, physics=False, label=tac.replace('-', ' ').title(), x=tactic_coords[tac_index][0], y=tactic_coords[tac_index][1], **gconf.tactic)
    tac_index += 1
    if prev:
        net3.add_edge(prev, tac)
        pass
    prev = tac
    
# Techniques

for tac in full_index:
    


# for tech in d.attack.techniques:
#     tac = d.attack.techniques[tech].tactics[0]
    
#     parent = d.attack.techniques[tech].get_parent()
    
#     if parent and parent not in net3.node_ids:
#         parent_tac = d.attack.techniques[parent].tactics[0]
#         net3.add_node(parent, label= d.attack.get_name_by_id(parent), physics=False, **gconf.technique)
#         #net.add_node(parent, color=gconf.technique['color'], mass=gconf.technique['mass'])
#         net3.add_edge(parent, parent_tac)
    
#     if parent:
#         net3.add_node(tech, label = d.attack.get_name_by_id(tech),  **gconf.subtechnique)
#         #net.add_node(tech, color=gconf.subtechnique['color'], mass=gconf.subtechnique['mass'])
#         net3.add_edge(tech, parent)
#     else:
#         net3.add_node(tech, label = d.attack.get_name_by_id(tech),physics=False, **gconf.technique)
#         #net.add_node(tech, color=gconf.technique['color'], mass=gconf.technique['mass'])
#         net3.add_edge(tech, tac)

    
net3.show('test2.html')

15
14
{'execution': 15, 'collection': 16, 'persistence': 14, 'credential-access': 16, 'discovery': 28, 'resource-development': 8, 'reconnaissance': 10, 'defense-evasion': 42, 'impact': 13, 'lateral-movement': 9, 'command-and-control': 17, 'initial-access': 6, 'exfiltration': 9, 'privilege-escalation': 4}
test2.html


In [208]:
full_index = {}
for tac in d.attack.cfg.tactics_order:
    full_index.update({tac: {}})
    
for tech in d.attack.techniques:
    if "." not in tech:
        tac = d.attack.techniques[tech].tactics[0]
        full_index[tac].update({tech: []})

for tech in d.attack.techniques:
    if "." in tech:
        parent = tech.split('.')[0]
        parent_tac = d.attack.techniques[parent].tactics[0]
        full_index[parent_tac][parent].append(tech)

In [232]:
net4 = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')

tac_rad = 8000
tech_rad = 2000
subtech_rad = 300

# Tactics
prev = None
tactic_coords = get_coords(tac_rad, x=0, y=0, minresults=len(d.attack.cfg.tactics_order)+1)
print(len(tactic_coords))
print(len(d.attack.cfg.tactics_order))
tac_index = 0

for tac in full_index:
    tac_x = tactic_coords[tac_index][0]
    tac_y = tactic_coords[tac_index][1]
    
    net4.add_node(tac, physics=False, label=tac.replace('-', ' ').title(), x=tac_x, y=tac_y, **gconf.tactic)
    tac_index += 1
    if prev:
        net4.add_edge(prev, tac)
        pass
    prev = tac
    
    # Techniques
    tech_index = 0
    technique_coords = get_coords(tech_rad, x=tac_x, y=tac_y, minresults=len(full_index[tac])+1)
    for tech in full_index[tac]:
        tech_x = technique_coords[tech_index][0]
        tech_y = technique_coords[tech_index][1]
        net4.add_node(tech, physics=False, label= d.attack.get_name_by_id(tech), x=tech_x, y=tech_y, **gconf.technique)
        tech_index += 1
        net4.add_edge(tech, tac)
        
        
        # Subtechniques
        subtech_index = 0
        subtechnique_coords = get_coords(subtech_rad, x=tech_x, y=tech_y, minresults=len(full_index[tac][tech])+1)
        for subtech in full_index[tac][tech]:
            subtech_x = subtechnique_coords[subtech_index][0]
            subtech_y = subtechnique_coords[subtech_index][1]
            net4.add_node(subtech, physics=True, label= d.attack.get_name_by_id(subtech), x=subtech_x, y=subtech_y, **gconf.subtechnique)
            subtech_index += 1
            net4.add_edge(subtech, tech)
            
for s in sigma_wtech:
    #net.add_node(s['id'], label=s['title'], color='#AAF')
    for t in s['techniques']:
        if t in net4.node_ids:
            if s['severity'] == 'high':
                color = sev_colors[s['severity']]
                net4.add_node(s['id']+t, label=s['title'], color=color, mass=1)
                net4.add_edge(s['id']+t, t)
        
net4.show('test3.html')

15
14
test3.html


In [ ]:
Ideas:
    
    Create inner or outer circles for sigma rules. 3 circles for each severity. generate multiple coordinates on those circles. Choose proper coords based on distance to mapped technique. Remember coords already taken to use the next one. 